<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Chatbot_using_LangChain_with_Memory_06_02_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install langchain_groq langchain langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.0 MB/s eta 0:00:00


In [32]:
# import os
# LANGSMITH_TRACING = os.environ.get('LANGSMITH_TRACING')
# LANGSMITH_ENDPOINT = os.environ.get('LANGSMITH_ENDPOINT')
# LANGSMITH_API_KEY = os.environ.get('LANGSMITH_API_KEY')
# LANGSMITH_PROJECT = os.environ.get('LANGSMITH_PROJECT')

In [29]:
from google.colab import userdata
LANGSMITH_TRACING = userdata.get('LANGSMITH_TRACING')
LANGSMITH_ENDPOINT = userdata.get('LANGSMITH_ENDPOINT')
LANGSMITH_API_KEY = userdata.get('LANGSMITH_API_KEY')
LANGSMITH_PROJECT = userdata.get('LANGSMITH_PROJECT')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [33]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192", api_key=GROQ_API_KEY)

model.invoke("hi").content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [34]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser.invoke(model.invoke("hi"))

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [35]:
from langchain_core.messages import HumanMessage

import warnings
warnings.filterwarnings("ignore")

In [36]:
while True:
  message = input("Write your query:")
  if message=="bye":
    print("Good Bye have a great day!")
    break
  else:
    print(parser.invoke(model.invoke([HumanMessage(content=message)])))

Write your query:hi
Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?
Write your query:bye
Good Bye have a great day!


In [38]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


from langchain_core.messages import AIMessage

In [39]:
result = model.invoke(
    [
        HumanMessage(content="Hi! I'm Preetam"),
        AIMessage(content="Hello Preetam! How can I assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)

In [40]:
parser.invoke(result)

'Your name is Preetam!'

# with memory

In [41]:
store={}

In [42]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [43]:
config = {"configurable": {"session_id": "firstchat"}}

In [44]:
model_with_memory=RunnableWithMessageHistory(model,get_session_history)

In [45]:
model_with_memory.invoke([HumanMessage(content="Hi! I'm Preetam")],config=config).content

'Nice to meet you, Preetam! How can I help you today?'

In [46]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'Your name is Preetam!'

In [47]:
config = {"configurable": {"session_id": "secondchat"}}

In [48]:
config

{'configurable': {'session_id': 'secondchat'}}

In [49]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

"I'm just an AI, I don't have the ability to know your name unless you've already told me or provided it to me."

In [50]:
config = {"configurable": {"session_id": "firstchat"}}

In [51]:
config

{'configurable': {'session_id': 'firstchat'}}

In [52]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Preetam", additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Preetam! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34, 'completion_time': 0.014166667, 'prompt_time': 0.002473329, 'queue_time': 0.278004639, 'total_time': 0.016639996}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-f147cf50-6e2b-44b4-80ee-daee045f1259-0', usage_metadata={'input_tokens': 17, 'output_tokens': 17, 'total_tokens': 34}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Preetam!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 48, 'total_tokens': 56, 'completion_time': 0.006666667, 'prompt_time': 0.018567644

In [53]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'I already told you, your name is Preetam!'

In [54]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [55]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [56]:
chain = prompt | model

In [58]:
chain.invoke({"messages": ["hi! I'm Preetam"]})

AIMessage(content="Hello Preetam! It's nice to meet you! I'm here to help you with any questions or tasks you may have. What's on your mind?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 38, 'total_tokens': 72, 'completion_time': 0.028333333, 'prompt_time': 0.007066561, 'queue_time': 0.26140843199999997, 'total_time': 0.035399894}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-161978d1-f34d-49e9-8202-d6d486d72286-0', usage_metadata={'input_tokens': 38, 'output_tokens': 34, 'total_tokens': 72})

In [59]:
chain.invoke({"messages": [HumanMessage(content="hi! I'm Preetam")]}).content

'Nice to meet you, Preetam! Hi! How can I help you today?'

In [60]:
chain.invoke({"messages": [HumanMessage(content="what is my name")]}).content

"I apologize, but I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to learn it and address you by name in our conversation!"

In [61]:
model_with_memory=RunnableWithMessageHistory(chain,get_session_history)

In [62]:
config = {"configurable": {"session_id": "thirdchat"}}

In [63]:
response=model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Preetam"),
     ],config=config
)

In [64]:
response.content

"Hi Preetam! It's nice to meet you! I'm here to help you with any questions or topics you'd like to discuss. What's on your mind today?"

In [65]:
response=model_with_memory.invoke(
    [HumanMessage(content="hi what is my name"),
     ],config=config
)
print(response.content)

I remember! Your name is Preetam!


In [66]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is 2+2?"),
     ],config=config
)
print(response.content)

Easy one! The answer to 2+2 is... 4!


In [67]:
response=model_with_memory.invoke(
    [HumanMessage(content="who is a indian cricket team caption?"),
     ],config=config
)
print(response.content)

A popular one! As of now, the captain of the Indian national cricket team is Virat Kohli.


In [68]:
response=model_with_memory.invoke(
    [HumanMessage(content="what should i add in previous addition so that i will become 10?"),
     ],config=config
)
print(response.content)

A math puzzle!

You said 2+2 is 4. To make it equal to 10, you need to add 6 to 4.

So, you would need to add 6 to 4 to get 10.


In [69]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is my name?"),
     ],config=config
)
print(response.content)

I remember! Your name is Preetam!


In [70]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Preetam", additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Preetam! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34, 'completion_time': 0.014166667, 'prompt_time': 0.002473329, 'queue_time': 0.278004639, 'total_time': 0.016639996}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-f147cf50-6e2b-44b4-80ee-daee045f1259-0', usage_metadata={'input_tokens': 17, 'output_tokens': 17, 'total_tokens': 34}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Preetam!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 48, 'total_tokens': 56, 'completion_time': 0.006666667, 'prompt_time': 0.018567644

# Trimming the conversation

In [71]:
from langchain_core.messages import SystemMessage, trim_messages

In [77]:
trimmer = trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [78]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [79]:
model.get_num_tokens_from_messages(messages)

47

In [80]:
trimmer.invoke(messages)

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [81]:
trimmed_message = trimmer.invoke(messages)

In [82]:
model.get_num_tokens_from_messages(trimmed_message)

26

In [83]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [84]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [85]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [86]:
messages

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [87]:
trimmed_message

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [88]:
trimmed_message+[HumanMessage(content="what's my name?")]

[HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [89]:
messages+[HumanMessage(content="what's my name?")]

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [99]:
trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [100]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [101]:
messages + [HumanMessage(content="what's my name?")]

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [102]:
trimmed_message = trimmer.invoke(messages)
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [103]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I apologize, but I don't have that information. I'm a new assistant and we just started talking, so I don't have any personal information about you."

In [104]:
messages + [HumanMessage(content="what math problem did i ask")]

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what math problem did i ask', additional_kwargs={}, response_metadata={})]

In [105]:
trimmed_message = trimmer.invoke(messages)
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [106]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2"!'

In [107]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [108]:
config = {"configurable": {"session_id": "fourthchat"}}

In [109]:
messages + [HumanMessage(content="whats my name?")]

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats my name?', additional_kwargs={}, response_metadata={})]

In [110]:
trimmed_message = trimmer.invoke(messages)
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [111]:
response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I apologize, but I don't know your name. I'm a new assistant and we just started chatting. You haven't told me your name yet!"

In [113]:
messages + [HumanMessage(content="what math problem did i ask?")]

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what math problem did i ask?', additional_kwargs={}, response_metadata={})]

In [114]:
trimmed_message = trimmer.invoke(messages)
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [115]:
response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked me what 2 + 2 is, and I told you the answer is 4!'

In [116]:
response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a math problem yet! This conversation just started, and you didn't mention any math problem. I'm here to help whenever you need it, though!"